
**Hola! Héctor**

Mi nombre es Enrique Romero, tendre el gusto de revisar Tu proyecto, te deseo lo mejor.

<div class="alert alert-danger">
<b>❌ Comentario del revisor:</b> Esto destaca los comentarios más importantes. Sin su desarrollo, el proyecto no será aceptado. </div>

<div class="alert alert-warning">
<b>⚠️ Comentario del revisor:</b> Así que los pequeños comentarios están resaltados. Se aceptan uno o dos comentarios de este tipo en el borrador, pero si hay más, deberá hacer las correcciones. Es como una tarea de prueba al solicitar un trabajo: muchos pequeños errores pueden hacer que un candidato sea rechazado.
</div>

<div class="alert alert-success">
<b>✔️ Comentario del revisor:</b> Así que destaco todos los demás comentarios.</div>

<div class="alert alert-info"> <b>Comentario del estudiante:</b> Por ejemplo, asi.</div>

Todo esto ayudará a volver a revisar tu proyecto más rápido.




## Inicialización

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import seaborn as sns

import tensorflow as tf

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout, Flatten
from tensorflow.keras.optimizers import Adam

## Cargar datos

El conjunto de datos se almacena en la carpeta `/datasets/faces/`
- La carpeta `final_files` con 7600 fotos
- El archivo `labels.csv` con leyendas, con dos columnas: `file_name` y `real_age`

Dado que el número de archivos de imágenes es bastante elevado, se recomienda evitar leerlos todos a la vez, ya que esto consumiría muchos recursos computacionales. Te recomendamos crear un generador con ImageDataGenerator. Este método se explicó en el capítulo 3, lección 7 de este curso.

El archivo de etiqueta se puede cargar como un archivo CSV habitual.

In [ ]:
try:
    labels = pd.read_csv('/datasets/faces/labels.csv')
except:
    labels = pd.read_csv('labels.csv')

<div class="alert alert-success">
<b>✔️ Comentario del revisor:</b>    
Excelente inicio, muy bien con esta exploración inicial
</div>	


In [ ]:
labels.info()

In [ ]:
labels.head()

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255)
    
train_gen_flow = train_datagen.flow_from_dataframe(dataframe=labels,
                                                  directory='/datasets/faces/final_files',
                                                  x_col='file_name',
                                                  y_col='real_age',
                                                  target_size=(224,224),
                                                  batch_size=32,
                                                  class_mode='raw',
                                                  seed=12345)


In [ ]:
features, target = next(train_gen_flow)

In [ ]:
features.shape

In [ ]:
fig = plt.figure(figsize=(16,16))
for i in range(15):
    fig.add_subplot(5,3,i+1)
    plt.imshow(features[i])
    plt.title(f'{target[i]}')
    plt.yticks([])
    plt.xticks([])
    plt.tight_layout()

## EDA

In [ ]:
labels['real_age'].describe()

In [ ]:
plt.figure(figsize=(10,6))
sns.set_style('whitegrid')
sns.set_palette('inferno')
sns.distplot(labels['real_age'], rug=True, kde_kws={'shade':True, 'color':'r'})
plt.xlabel('Edad', fontsize=12, color='b')
plt.ylabel('Frecuencia', fontsize=12, color='b')
plt.title('Distribución de edades', fontsize=16, color='b')
plt.show()

In [ ]:
labels.boxplot('real_age', figsize=(16,9))

In [ ]:
under_age = labels[labels['real_age']<21].shape[0]
print(f'El {(under_age/labels.shape[0])*100:.2f} % del total de las personas son menores de edad')

### Conclusiones

- Tenemos un grupo del 73% de personas que tienen la mayoría de edad en el conjunto de datos.
- Hay personas de más de 75 años que para nuestro caso podríamos dejar de lado.
- El promedio ronda los 31 años de edad

<div class="alert alert-success">
<b>✔️ Comentario del revisor:</b>    
Excelente inicio, muy bien con las conclusiones parciales   
</div>	

## Modelado

Define las funciones necesarias para entrenar tu modelo en la plataforma GPU y crear un solo script que las contenga todas junto con la sección de inicialización.

Para facilitar esta tarea, puedes definirlas en este notebook y ejecutar un código listo en la siguiente sección para componer automáticamente el script.

Los revisores del proyecto también verificarán las definiciones a continuación, para que puedan comprender cómo construiste el modelo.

In [ ]:
def load_train(path):
    
    """
    Carga la parte de entrenamiento del conjunto de datos desde la ruta.
    """
    
    # coloca tu código aquí
    df = pd.read_csv(path + 'labels.csv')
    directory = path + 'final_files/'
    
    train_datagen = ImageDataGenerator(rescale=1./255)
    
    train_gen_flow = train_datagen.flow_from_dataframe(dataframe=df,
                                                    directory=directory,
                                                    x_col='file_name',
                                                    y_col='real_age',
                                                    target_size=(224,224),
                                                    batch_size=32,
                                                    class_mode='raw',
                                                    seed=12345)

   
    return train_gen_flow

In [ ]:
def load_test(path):
    
    """
    Carga la parte de validación/prueba del conjunto de datos desde la ruta
    """
    
    #  coloca tu código aquí
    df = pd.read_csv(path + 'labels.csv')
    directory = path + 'final_files/'
    
    train_datagen = ImageDataGenerator(rescale=1./255)
    
    test_gen_flow = train_datagen.flow_from_dataframe(dataframe=df,
                                                    directory=directory,
                                                    x_col='file_name',
                                                    y_col='real_age',
                                                    target_size=(224,224),
                                                    batch_size=32,
                                                    class_mode='raw',
                                                    seed=12345)

    return test_gen_flow

In [ ]:
def create_model(input_shape=(224,224,3)):
    
    """
    Define el modelo
    """
    
    #  coloca tu código aquí
    backbone=ResNet50(input_shape=input_shape,
                      weights='imagenet',
                      include_top=False)
    
    model=Sequential()
    model.add(backbone)
    model.add(GlobalAveragePooling2D())
    model.add(Dense(24,activation='relu'))
    model.add(Dense(24, activation='relu'))
    
    model.compile(optimizer=Adam(learning_rate=0.001),
                  loss='tf.keras.losses.MeanAbsoluteError()',
                  metrics=[tf.keras.metrics.MeanAbsoluteError()])

    return model

In [ ]:
def train_model(model, train_data, test_data, batch_size=None, epochs=20,
                steps_per_epoch=None, validation_steps=None):

    """
    Entrena el modelo dados los parámetros
    """
    
    #  coloca tu código aquí
    if steps_per_epoch is None:
        steps_per_epoch=len(train_data)
        
    if validation_steps is None:
        validation_steps=len(test_data)
        
    model.fit(train_data,
              validation_steps=validation_steps,
              steps_per_epoch=steps_per_epoch,
              validation_data=test_data,
              batch_size=batch_size,
              epochs=epochs,
              verbose=2)
    

    return model

<div class="alert alert-success">
<b>✔️ Comentario del revisor:</b>    
Muy bien con la implementacion de las funciones, esto beneficia el procesamiento y la eficiencia del proyecto
</div>	

### Prepara el script para ejecutarlo en la plataforma GPU

Una vez que hayas definido las funciones necesarias, puedes redactar un script para la plataforma GPU, descargarlo a través del menú "File|Open..." (Archivo|Abrir) y cargarlo más tarde para ejecutarlo en la plataforma GPU.

Nota: el script debe incluir también la sección de inicialización. A continuación se muestra un ejemplo.

In [ ]:
# prepara un script para ejecutarlo en la plataforma GPU

init_str = """
import pandas as pd

import tensorflow as tf

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout, Flatten
from tensorflow.keras.optimizers import Adam

train_data = load_train
test_data = load_test

model = create_model(input_shape=(224,224,3))

model = train_model(model, train_data, test_data)
"""

import inspect

with open('run_model_on_gpu.py', 'w') as f:
    
    f.write(init_str)
    f.write('\n\n')
        
    for fn_name in [load_train, load_test, create_model, train_model]:
        
        src = inspect.getsource(fn_name)
        f.write(src)
        f.write('\n\n')

### Resultado

Coloca aquí el resultado de la plataforma GPU como una celda Markdown.

<class 'tensorflow.python.keras.engine.sequential.Sequential'>

Train for 238 steps, validate for 238 steps
Epoch 1/20
2024-02-15 03:40:17.886333: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10
2024-02-15 03:40:18.618960: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudnn.so.7
238/238 - 92s - loss: 16.1741 - mean_absolute_error: 16.1784 - val_loss: 19.1979 - val_mean_absolute_error: 19.1905

Epoch 2/20
238/238 - 74s - loss: 14.4509 - mean_absolute_error: 14.4470 - val_loss: 18.9442 - val_mean_absolute_error: 18.9794

Epoch 3/20
238/238 - 74s - loss: 13.3164 - mean_absolute_error: 13.3128 - val_loss: 15.5322 - val_mean_absolute_error: 15.5353

Epoch 4/20
238/238 - 74s - loss: 12.2795 - mean_absolute_error: 12.2774 - val_loss: 13.3639 - val_mean_absolute_error: 13.3805

Epoch 5/20
238/238 - 74s - loss: 11.7189 - mean_absolute_error: 11.7100 - val_loss: 11.8357 - val_mean_absolute_error: 11.8453

Epoch 6/20
238/238 - 75s - loss: 11.2444 - mean_absolute_error: 11.2415 - val_loss: 13.9537 - val_mean_absolute_error: 13.9785

Epoch 7/20
238/238 - 74s - loss: 10.2692 - mean_absolute_error: 10.2649 - val_loss: 10.5515 - val_mean_absolute_error: 10.5582

Epoch 8/20
238/238 - 74s - loss: 9.5517 - mean_absolute_error: 9.5393 - val_loss: 9.7000 - val_mean_absolute_error: 9.7062

Epoch 9/20
238/238 - 73s - loss: 8.9809 - mean_absolute_error: 8.9819 - val_loss: 11.0880 - val_mean_absolute_error: 11.0966

Epoch 10/20
238/238 - 74s - loss: 8.5641 - mean_absolute_error: 8.5718 - val_loss: 11.8971 - val_mean_absolute_error: 11.9179

Epoch 11/20
238/238 - 74s - loss: 8.4118 - mean_absolute_error: 8.4111 - val_loss: 10.8473 - val_mean_absolute_error: 10.8567

Epoch 12/20
238/238 - 74s - loss: 7.9976 - mean_absolute_error: 8.0053 - val_loss: 8.9715 - val_mean_absolute_error: 8.9813

Epoch 13/20
238/238 - 74s - loss: 7.7648 - mean_absolute_error: 7.7689 - val_loss: 8.0999 - val_mean_absolute_error: 8.1099

Epoch 14/20
238/238 - 73s - loss: 7.3701 - mean_absolute_error: 7.3735 - val_loss: 8.5230 - val_mean_absolute_error: 8.5316

Epoch 15/20
238/238 - 74s - loss: 6.2939 - mean_absolute_error: 6.2970 - val_loss: 6.7856 - val_mean_absolute_error: 6.7918

Epoch 16/20
238/238 - 74s - loss: 6.0679 - mean_absolute_error: 6.0755 - val_loss: 8.5609 - val_mean_absolute_error: 8.5713

Epoch 17/20
238/238 - 73s - loss: 4.7823 - mean_absolute_error: 4.7878 - val_loss: 9.8039 - val_mean_absolute_error: 9.8194

Epoch 18/20
238/238 - 74s - loss: 4.6361 - mean_absolute_error: 4.6392 - val_loss: 5.5171 - val_mean_absolute_error: 5.5249

Epoch 19/20
238/238 - 74s - loss: 4.4585 - mean_absolute_error: 4.4639 - val_loss: 6.8342 - val_mean_absolute_error: 6.8436

Epoch 20/20
238/238 - 74s - loss: 4.2171 - mean_absolute_error: 4.2205 - val_loss: 6.0806 - val_mean_absolute_error: 6.0917

238/238 - 37s - loss: 6.0806 - mean_absolute_error: 6.0917

Test MAE: 6.0917


## Conclusiones

En este proyecto se trabajo con una red neuronal ResNet en la que se busca determinar la edad de los clientes de una tienda para evitar que menores compren alcohol.
El modelo presentado obtuvo un valor adecuado de error medio absoluto menor a 7, lo que es lo que buscamos en este trabajo obteniendo un 6.09. Esta red neuronal podemos afirmar que puede ayudar a la tienda a comprobar la edad de los clientes que llegan a mostrador.
Por otra parte este mismo modelo puede tener otra aplicación para la entrada a bares y espectaculos que tienen limite de edad, como películas, obras de teatro o exposiciones de arte.

# Lista de revisión

- [ ]  El Notebook estaba abierto 
- [ ]  El código no tiene errores
- [ ]  Las celdas con el código han sido colocadas en el orden de ejecución
- [ ]  Se realizó el análisis exploratorio de datos
- [ ]  Los resultados del análisis exploratorio de datos se presentan en el notebook final
- [ ]  El valor EAM del modelo no es superior a 8
- [ ]  El código de entrenamiento del modelo se copió en el notebook final
- [ ]  El resultado de entrenamiento del modelo se copió en el notebook final
- [ ]  Los hallazgos se proporcionaron con base en los resultados del entrenamiento del modelo

<div class="alert alert-success">
<b>✔️ Comentario del revisor:</b>
<h1>    Comentarios Generales </h1>
Felicidades has aprobado Héctor        
</div>